In [1]:
# stp 1 Dependecis instal
#!pip install transformers==4.41.2
# Important to install that transformer version, unles it occur error
#See link: https://github.com/THUDM/CogVLM2/issues/181#issuecomment-2381807778
# !pip install transformers sentencepiece datasets evaluate
# !pip install llmlingua

In [1]:
import torch
import json
import time
import os
from transformers import AutoModel, AutoTokenizer, GenerationConfig
from datasets import load_dataset
from tqdm.notebook import tqdm 
import numpy as np
from sklearn.metrics import f1_score
#from LLMLingua.experiments.llmlingua2.evaluation.metrics import qa_f1_score
from metrics import qa_f1_score, f1_score
if 'model' in globals():
    del model 
torch.cuda.empty_cache()

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#######################################
print('cuda is',torch.cuda.is_available()) 
#################################################
# 3 model & dataset 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuda is True


In [2]:
print('cuda is',torch.cuda.is_available()) 

cuda is True


In [3]:
# ChatGLM-6B
model = AutoModel.from_pretrained(
    "THUDM/chatglm2-6b-32k",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True
).to(device)

model=model.eval()
#print('model config is:',model.generation_config)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b-32k", trust_remote_code=True)#.to(device)

In [5]:
# Monkey-patch the tokenizer's _pad method to handle the padding_side argument
original_pad = tokenizer._pad
def new_pad(self, encoded_inputs, max_length=None, padding_strategy="longest", pad_to_multiple_of=None, return_attention_mask=None, **kwargs):
    return original_pad(encoded_inputs, max_length, padding_strategy, pad_to_multiple_of, return_attention_mask)
tokenizer._pad = new_pad.__get__(tokenizer, type(tokenizer))


In [6]:
# Load the  four datasets
datasets = ["multifieldqa_en", "hotpotqa", "triviaqa", "narrativeqa"]

In [7]:
# LINGUA COMPRESSION

from llmlingua import PromptCompressor
compressor = PromptCompressor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
    device_map="cuda",  # Use GPU
    use_llmlingua2=True,
)

/home/seongjae/miniconda3/envs/seongjae_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
def evaluate(data, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []
    
    # Model constraints
    max_context_length = 32768  # ChatGLM2-6B-32k's limit
    max_new_tokens = 256  # Allow generating up to 256 tokens
    max_input_length = max_context_length - max_new_tokens  # 32768 - 256 = 32512
    
    #for idx in range(20):  # Test on 10 examples
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        #print(type(example))
        #print(example)
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]

        compressed_context = compressor.compress_prompt(
            context,
            rate=ratio,           # Keep 50% tokens (compression ratio)
        )["compressed_prompt"]
        #print('compressed_context:',compressed_context)

        tokenized = tokenizer.encode(
            compressed_context, 
            truncation=True, 
            max_length=max_input_length
        )
        truncated_context = tokenizer.decode(tokenized, skip_special_tokens=True)

        torch.cuda.reset_peak_memory_stats()        
        start_time = time.time()
        response, _ = model.chat(
            tokenizer,
            question,
            history=[(compressed_context, "Answer:")],
            max_new_tokens=max_new_tokens  # Critical parameter
        )
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        peak_memory_usages.append(peak_memory)
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [11]:
# Reusable function to print and save results
        
def log_result_LLMLingua(message):
    #print(message)
    with open("all_results_LLMLingua.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

In [11]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[0]  #multifiledqa_en
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
for ratio in pruning_ratios:
    # lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    lingua_results = evaluate(data, ratio=ratio)
    
    print(f"\n==={dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n===20 samples {dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")

Token indices sequence length is longer than the specified maximum sequence length for this model (1237 > 512). Running this sequence through the model will result in indexing errors



===multifieldqa_en  llmlingua compression (25%) ===
F1: 0.20, Latency: 3.44s,  Peak Memory: 14415.18 MB

===multifieldqa_en  llmlingua compression (50%) ===
F1: 0.30, Latency: 2.11s,  Peak Memory: 14738.71 MB

===multifieldqa_en  llmlingua compression (75%) ===
F1: 0.36, Latency: 2.37s,  Peak Memory: 15068.07 MB


In [12]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[1]  #hotpotqa
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
for ratio in pruning_ratios:
    # lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    lingua_results = evaluate(data, ratio=ratio)
    
    print(f"\n==={dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n===20 samples {dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")


===hotpotqa  llmlingua compression (25%) ===
F1: 0.15, Latency: 1.14s,  Peak Memory: 14740.71 MB

===hotpotqa  llmlingua compression (50%) ===
F1: 0.21, Latency: 1.77s,  Peak Memory: 15337.76 MB

===hotpotqa  llmlingua compression (75%) ===
F1: 0.26, Latency: 2.28s,  Peak Memory: 15926.03 MB


In [13]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[2]  #triviaqa
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
for ratio in pruning_ratios:
    # lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    lingua_results = evaluate(data, ratio=ratio)
    
    print(f"\n==={dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n===20 samples {dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")


===triviaqa  llmlingua compression (25%) ===
F1: 0.41, Latency: 0.72s,  Peak Memory: 14767.27 MB

===triviaqa  llmlingua compression (50%) ===
F1: 0.40, Latency: 1.22s,  Peak Memory: 15279.17 MB

===triviaqa  llmlingua compression (75%) ===
F1: 0.42, Latency: 1.85s,  Peak Memory: 15787.47 MB


In [ ]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[3]  #narrativeqa
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
for ratio in pruning_ratios:
    # lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    lingua_results = evaluate(data, ratio=ratio)
    
    print(f"\n==={dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n===20 samples {dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")